In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_csv('sokak_hayvanlari_veriseti.csv')

In [4]:
X = df[['Sokakta_Yasayan_Insan_Sayisi', 'Sokagin_Yuz_Olcumu', 'Sokaktaki_Gonullu_Sayisi',
        'Sokaktaki_Hayvanseverlik_Yuzdesi', 'Sokakta_Hayvanlar_Icin_Park_Bahce_Vb']]
y_kedi = df['Kedi_Sayisi']
y_kopek = df['Kopek_Sayisi']


In [5]:
X_train, X_test, y_kedi_train, y_kedi_test, y_kopek_train, y_kopek_test = train_test_split(X, y_kedi, y_kopek, test_size=0.2, random_state=42)


In [6]:
regressors = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42),
    'SVR': SVR()
}


In [7]:
results = {}
for name, reg in regressors.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', reg)
    ])
    pipeline.fit(X_train, y_kedi_train)
    y_kedi_pred = pipeline.predict(X_test)

    mse = mean_squared_error(y_kedi_test, y_kedi_pred)
    mae = mean_absolute_error(y_kedi_test, y_kedi_pred)
    r2 = r2_score(y_kedi_test, y_kedi_pred)

    results[name] = {
        'MSE': mse,
        'MAE': mae,
        'R-squared': r2
    }

In [8]:
for name, metrics in results.items():
    print(f'{name}:')
    print(f"  MSE: {metrics['MSE']}")
    print(f"  MAE: {metrics['MAE']}")
    print(f"  R-squared: {metrics['R-squared']}")
    print()

Linear Regression:
  MSE: 2.260107417682373
  MAE: 1.1939670467692398
  R-squared: 0.6986553979343177

Decision Tree:
  MSE: 1.986
  MAE: 0.842
  R-squared: 0.735202683279476

Random Forest:
  MSE: 0.8619144
  MAE: 0.60496
  R-squared: 0.885079244530322

SVR:
  MSE: 0.6054103396971083
  MAE: 0.512526856871424
  R-squared: 0.9192794393420669



In [9]:
best_model_kedi = min(results, key=lambda x: results[x]['MSE'])
print(f"En iyi model (kedi): {best_model_kedi}")


En iyi model (kedi): SVR


In [10]:
best_pipeline_kedi = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', regressors[best_model_kedi])
])
best_pipeline_kedi.fit(X_train, y_kedi_train)


Pipeline(steps=[('scaler', StandardScaler()), ('regressor', SVR())])

In [11]:
results_kopek = {}
for name, reg in regressors.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', reg)
    ])
    pipeline.fit(X_train, y_kopek_train)
    y_kopek_pred = pipeline.predict(X_test)

    mse = mean_squared_error(y_kopek_test, y_kopek_pred)
    mae = mean_absolute_error(y_kopek_test, y_kopek_pred)
    r2 = r2_score(y_kopek_test, y_kopek_pred)

    results_kopek[name] = {
        'MSE': mse,
        'MAE': mae,
        'R-squared': r2
    }


In [12]:
for name, metrics in results_kopek.items():
    print(f'{name}:')
    print(f"  MSE: {metrics['MSE']}")
    print(f"  MAE: {metrics['MAE']}")
    print(f"  R-squared: {metrics['R-squared']}")
    print()


Linear Regression:
  MSE: 1.1830581340962296
  MAE: 0.8720852072977353
  R-squared: 0.8733938358245702

Decision Tree:
  MSE: 2.92
  MAE: 1.312
  R-squared: 0.6875132432315583

Random Forest:
  MSE: 1.5060788
  MAE: 0.9720799999999999
  R-squared: 0.838825452174758

SVR:
  MSE: 1.2371258648211316
  MAE: 0.875076741289972
  R-squared: 0.8676077228724969



In [13]:
best_model_kopek = min(results_kopek, key=lambda x: results_kopek[x]['MSE'])
print(f"En iyi model (köpek): {best_model_kopek}")


En iyi model (köpek): Linear Regression


In [14]:
best_pipeline_kopek = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', regressors[best_model_kopek])
])
best_pipeline_kopek.fit(X_train, y_kopek_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor', LinearRegression())])

In [15]:
def make_prediction(insan_sayisi, yuz_olcumu, gonullu_sayisi, hayvanseverlik_yuzdesi, park_bahce):
    input_data = pd.DataFrame({
        'Sokakta_Yasayan_Insan_Sayisi': [insan_sayisi],
        'Sokagin_Yuz_Olcumu': [yuz_olcumu],
        'Sokaktaki_Gonullu_Sayisi': [gonullu_sayisi],
        'Sokaktaki_Hayvanseverlik_Yuzdesi': [hayvanseverlik_yuzdesi],
        'Sokakta_Hayvanlar_Icin_Park_Bahce_Vb': [park_bahce]
    })

    kedi_tahmin = best_pipeline_kedi.predict(input_data)[0]
    kopek_tahmin = best_pipeline_kopek.predict(input_data)[0]

    return kedi_tahmin, kopek_tahmin

In [16]:
insan_sayisi = 100
yuz_olcumu = 5.0
gonullu_sayisi = 20
hayvanseverlik_yuzdesi = 80
park_bahce = 1

kedi_tahmin, kopek_tahmin = make_prediction(insan_sayisi, yuz_olcumu, gonullu_sayisi, hayvanseverlik_yuzdesi, park_bahce)
print(f"Kedi tahmini: {kedi_tahmin}")
print(f"Köpek tahmini: {kopek_tahmin}")

Kedi tahmini: 12.33406764428029
Köpek tahmini: 12.210178824342048
